# Build a L-layer Neural Network.ipynb

## Table of Contents

* [Recap](#chapter0)
* [1. L-laers Neural Network Model](#chapter1)
    * [1.1 Functions of our L-layers Neural Network ](#section_1_1)
        * [1.1.1 ](#section_1_1_1)
        * [1.1.2 ](#section_1_1_2)
        * [1.1.3 ](#section_1_1_3)
        * [1.1.4 ](#section_1_1_4)
        * [1.1.5 ](#section_1_1_5)
    * [1.2 Model](#section_1_2)
* [2. Load the Dataset ](#chapter2)
    * [2.1 Load the Dataset](#section_2_1)
    * [2.2 Display the Data](#section_2_2)
    * [2.3 Flatten the data](#section_2_3)
    * [2.4 Normalize the data](#section_2_4)

# Recap  <a class="anchor" id="chapter0"></a>

> Forward Propagation :

<center><img src="images/05-Deep Neural network/forward-prop.png" width = "600px"></center>

$$
\begin{cases}
    Z^{[l]} = W^{[l]} X + b^{[l]} \\
    A^{[l]} = g^{[l]}(Z^{[l]}) 
\end{cases}
$$


> Backward Propagation :

<center><img src="images/05-Deep Neural network/backward-prop.png" width = "600px"></center>

$$
\begin{cases}
    dZ^{[l]} =  (A^{[l]} - Y) \\
    dW^{[l]} = \frac{1}{m} dZ^{[l]}A^{[l-1]T} \\
    db^{[l]} = \frac{1}{m} \sum dZ^{[l]}    \\
    dA^{[l-1]} =W^{[l]T}dZ^{[l]} \\
    dZ^{[l-1]} = W^{[l]T}dZ^{[l]} * g^{[l-1]'}(Z^{[l-1]}) \\
\end{cases}
$$

$$
\begin{cases}
    dZ^{[1]} = W^{[2]T}dZ^{[2]} * g^{[1]'}(Z^{[1]}) \\
    dW^{[1]} = \frac{1}{m} dZ^{[1]} X^T \\
    db^{[1]} = \frac{1}{m} \sum  dZ^{[1]}
\end{cases}
$$


> Dimension :
- m : number of examples

$$\begin{cases}
    Z^{[l]},A^{[l]} : (n^{[l]},m) \\
    W^{[l]}: (n^{[l]},n^{[l-1]})   \\
    b^{[l]}: (n^{[l]},1)  \\
    dZ^{[l]},dA^{[l]} : (n^{[l]},m) \\
    dW^{[l]} : (n^{[l]},n^{[l-1]})   \\
    db^{[l]}: (n^{[l]},1)
\end{cases}
$$


# 1. L-layers Neural Network model <a class="anchor" id="chapter1"></a>

## 1.1 Functions of our L-layers Neural Network  <a class="anchor" id="section_1_1"></a>

### 1.1.1 Initialize parameters  <a class="anchor" id="section_1_1_1"></a>

### 1.1.2 Forward propagation  <a class="anchor" id="section_1_1_2"></a>

### 1.1.3 Cost function  <a class="anchor" id="section_1_1_3"></a>

### 1.1.4 Backward Propagation  <a class="anchor" id="section_1_1_4"></a>

### 1.1.5 Update parameters  <a class="anchor" id="section_1_1_5"></a>